**นายพชรพล เกตุแก้ว รหัสนักศึกษา 6610110190**
# Food Review Sentiment Classifier (NLP)
- โมเดลจำแนกความรู้สึกจากบทวิจารณ์อาหาร (เชิงบวก/เชิงลบ)

# ขั้นตอนที่ 1 ดาวน์โหลดและตั้งค่าชุดข้อมูล

In [ ]:
# เชื่อมต่อกับ Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ติดตั้งไลบรารี Kaggle
!pip install kaggle

# อัปโหลดไฟล์ kaggle.json
from google.colab import files
files.upload()

# ตั้งค่าไฟล์การที่อัปโหลด
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# ดาวน์โหลดชุดข้อมูล Amazon Fine Food Reviews จาก Kaggle
!kaggle datasets download -d snap/amazon-fine-food-reviews

# แตกไฟล์ zip ของชุดข้อมูลที่ดาวน์โหลดมา
!unzip amazon-fine-food-reviews.zip

ในส่วนของขั้นตอนนี้ทำหน้าที่  
- เชื่อมต่อกับ Google Drive เพื่อให้สามารถบันทึกไฟล์ผลลัพธ์ (เช่น โมเดลหรือ tokenizer) ลงในไดรฟ์ได้ในภายหลัง  
- ติดตั้งไลบรารี `kaggle` ที่จำเป็นสำหรับการดาวน์โหลดชุดข้อมูลจากแพลตฟอร์ม Kaggle  
- อัปโหลดไฟล์ `kaggle.json` ซึ่งเป็นไฟล์รับรองความถูกต้อง (API token) ที่ใช้ในการยืนยันตัวตนกับ Kaggle  
- ตั้งค่าสิทธิ์การเข้าถึงไฟล์ `kaggle.json` อย่างปลอดภัยด้วยคำสั่ง chmod เพื่อป้องกันข้อผิดพลาดด้านสิทธิ์  
- ดาวน์โหลดชุดข้อมูล **Amazon Fine Food Reviews** จาก Kaggle โดยใช้คำสั่ง `kaggle datasets download`  
- แตกไฟล์ zip ที่ได้รับมาเพื่อให้สามารถอ่านไฟล์ CSV ด้วย pandas ได้ในขั้นตอนถัดไป

# ขั้นตอนที่ 2 นำเข้าไลบารี่ต่างๆที่จำเป็น

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

# ขั้นตอนที่ 3 เตรียมและประมวลผลข้อมูล

In [ ]:
# โหลดข้อมูลรีวิวอาหาร 50,000 แถวแรกจากไฟล์ CSV
df = pd.read_csv('Reviews.csv', nrows=50000)

# ฟังก์ชันทำความสะอาดข้อความ: แปลงเป็นตัวพิมพ์เล็ก ลบอักขระที่ไม่ใช่ตัวอักษรหรือช่องว่าง และจัดรูปแบบช่องว่าง
def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# เตรียมข้อมูล: เลือกเฉพาะคอลัมน์ 'Text' และ 'Score', ลบแถวที่มีค่าหายไป
df = df[['Text', 'Score']].dropna()

# ทำความสะอาดข้อความในคอลัมน์ 'Text'
df['Text'] = df['Text'].apply(clean_text)

# ลบแถวที่ข้อความว่างเปล่าหลังการทำความสะอาด
df = df[df['Text'] != ""]

# ลบข้อมูลซ้ำโดยพิจารณาจากข้อความรีวิว
df = df.drop_duplicates(subset=['Text'])

# ตัดรีวิวที่ให้คะแนนเป็น 3 (เป็นกลาง) ออก เพื่อเน้นการจำแนกเชิงบวก/เชิงลบ
df = df[df['Score'] != 3]

# สร้างป้ายกำกับความรู้สึก: 1 = บวก (คะแนน > 3), 0 = ลบ (คะแนน < 3)
df['sentiment'] = df['Score'].apply(lambda x: 1 if x > 3 else 0)

# แยกข้อมูลข้อความ (X) และป้ายกำกับ (y) สำหรับการฝึกโมเดล
X = df['Text'].values
y = df['sentiment'].values

ในส่วนของขั้นตอนนี้ทำหน้าที่  
- โหลดข้อมูลรีวิวอาหาร 50,000 แถวแรกจากไฟล์ `Reviews.csv` เพื่อจำกัดขนาดข้อมูล
- สร้างฟังก์ชัน `clean_text()` สำหรับทำความสะอาดข้อความ โดยแปลงตัวอักษรเป็นพิมพ์เล็ก ลบสัญลักษณ์หรือตัวเลขที่ไม่ใช่ตัวอักษรภาษาอังกฤษ และจัดระเบียบช่องว่างให้เรียบร้อย  
- คัดเลือกเฉพาะคอลัมน์ที่จำเป็น (`Text` และ `Score`) แล้วลบแถวที่มีค่าหายไป  
- ทำความสะอาดข้อความทุกแถว ลบข้อความว่างเปล่า และกำจัดข้อมูลซ้ำเพื่อป้องกันการเรียนรู้ซ้ำซ้อน  
- ตัดรีวิวที่ให้คะแนนเป็นกลาง (คะแนน = 3) ออก เนื่องจากต้องการจำแนกเฉพาะความรู้สึก “เชิงบวก” หรือ “เชิงลบ”  
- แปลงคะแนนรีวิวให้เป็นป้ายกำกับไบนารี: 1 สำหรับรีวิวเชิงบวก (คะแนน 4–5) และ 0 สำหรับรีวิวเชิงลบ (คะแนน 1–2)  
- แยกข้อมูลข้อความ (`X`) และป้ายกำกับ (`y`) ออกมาเป็นอาร์เรย์ เพื่อเตรียมใช้ในการฝึกโมเดล NLP ต่อไป

In [ ]:
# แบ่งข้อมูลเป็นชุดฝึก (80%) และชุดทดสอบ (20%) โดยรักษาสัดส่วนของคลาส
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# กำหนดพารามิเตอร์สำหรับการแปลงข้อความ: จำนวนคำสูงสุดและความยาวลำดับ
MAX_WORDS = 20000
MAX_LEN = 120

# สร้างและฝึก Tokenizer บนชุดฝึก พร้อมจัดการคำที่ไม่เคยเห็นด้วย <OOV>
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# แปลงข้อความเป็นลำดับตัวเลข (sequences) สำหรับทั้งชุดฝึกและชุดทดสอบ
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# ปรับความยาวลำดับให้เท่ากันด้วยการเติมหรือตัดท้าย (padding/truncating)
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')

ในส่วนของขั้นตอนนี้ทำหน้าที่  
- แบ่งข้อมูลออกเป็นชุดฝึก (80%) และชุดทดสอบ (20%) โดยใช้ `train_test_split` พร้อมรักษาสัดส่วนของคลาส (เชิงบวก/เชิงลบ) ให้สมดุลด้วยพารามิเตอร์ `stratify=y` และกำหนด `random_state=42` เพื่อให้ผลลัพธ์สามารถทำซ้ำได้  
- กำหนดพารามิเตอร์สำหรับการประมวลผลข้อความ: จำนวนคำสูงสุดในพจนานุกรม (`MAX_WORDS = 20000`) และความยาวสูงสุดของลำดับข้อความ (`MAX_LEN = 120`)  
- สร้างและฝึก `Tokenizer` บนชุดฝึก เพื่อแปลงข้อความเป็นตัวเลข โดยใช้คำสูงสุดไม่เกิน `MAX_WORDS` และแทนคำที่ไม่อยู่ในพจนานุกรมด้วยโทเค็นพิเศษ `<OOV>`  
- แปลงข้อความในชุดฝึกและชุดทดสอบให้เป็นลำดับตัวเลข (sequences) ด้วย `texts_to_sequences()`  
- ปรับความยาวของทุกลำดับให้เท่ากันด้วย `pad_sequences()` โดยเติมศูนย์ด้านหลัง (`padding='post'`) และตัดข้อความที่ยาวเกิน `MAX_LEN` ออก (`truncating='post'`) เพื่อให้ข้อมูลพร้อมสำหรับการป้อนเข้าโมเดล NLP

# ขั้นตอนที่ 4 สร้างและฝึกโมเดล Bi-LSTM



In [ ]:
# คำนวณน้ำหนักคลาสเพื่อจัดการกับความไม่สมดุลของข้อมูล (class imbalance)
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

# สร้างโมเดล Sequential ด้วย Embedding, Bi-LSTM และ Dense layers สำหรับจำแนกความรู้สึกแบบไบนารี
model = Sequential([
    Embedding(input_dim=MAX_WORDS, output_dim=128, input_length=MAX_LEN),
    Bidirectional(LSTM(64, dropout=0.4, recurrent_dropout=0.4)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# คอมไพล์โมเดลด้วย optimizer Adam, loss แบบ binary crossentropy และวัดค่า accuracy
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# กำหนด callbacks: หยุดฝึกหากไม่ดีขึ้น (EarlyStopping) และลด learning rate เมื่อค่า loss ติด plateau
early_stop = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7)

# ฝึกโมเดลด้วยข้อมูลชุดฝึก พร้อมตรวจสอบกับชุดทดสอบ และใช้ class weights ชดเชยความไม่สมดุล
history = model.fit(
    X_train_pad, y_train,
    batch_size=128,
    epochs=15,
    validation_data=(X_test_pad, y_test),
    class_weight=class_weight_dict,
    callbacks=[early_stop, reduce_lr]
)

ในส่วนของขั้นตอนนี้ทำหน้าที่  
- คำนวณ **น้ำหนักของแต่ละคลาส** โดยใช้ `compute_class_weight` เพื่อชดเชยความไม่สมดุลระหว่างรีวิวเชิงบวกและเชิงลบ แล้วจัดเก็บเป็นพจนานุกรมสำหรับใช้ในระหว่างการฝึก  
- สร้าง **โมเดล Deep Learning แบบ Sequential** ประกอบด้วยชั้น Embedding สำหรับแปลงคำเป็นเวกเตอร์, ชั้น Bi-LSTM สำหรับจับลำดับบริบทจากทั้งสองทิศทาง, ตามด้วย Dropout เพื่อลด overfitting และชั้น Dense สุดท้ายสำหรับทำนายความน่าจะเป็นแบบไบนารี  
- คอมไพล์โมเดลด้วย **optimizer Adam** และ **loss function แบบ binary crossentropy** ซึ่งเหมาะกับงานจำแนกสองคลาส  
- กำหนด **callbacks** สองตัว: `EarlyStopping` เพื่อหยุดฝึกอัตโนมัติหากค่า validation loss ไม่ดีขึ้นภายใน 4 epochs และ `ReduceLROnPlateau` เพื่อลด learning rate เมื่อการเรียนรู้เริ่มติดจุดนิ่ง  
- ฝึกโมเดลด้วย **batch size 128**, สูงสุด 15 epochs โดยใช้ข้อมูลที่เตรียมไว้ (`X_train_pad`, `y_train`) และตรวจสอบผลกับชุดทดสอบแบบ real-time พร้อมใช้ `class_weight_dict` เพื่อให้โมเดลให้ความสำคัญกับคลาสที่มีข้อมูลน้อยมากขึ้น

# ขั้นตอนที่ 5 ประเมินผลโมเดล

In [ ]:
# ทำนายผลลัพธ์จากชุดทดสอบ และแปลงความน่าจะเป็นให้เป็นป้ายกำกับไบนารี (0 หรือ 1)
y_pred = (model.predict(X_test_pad) > 0.5).astype("int32")

# แสดงรายงานการจำแนก (Precision, Recall, F1-score) แยกตามคลาส 'Negative' และ 'Positive'
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))

# สร้างและแสดง Confusion Matrix แบบ Heatmap เพื่อวิเคราะห์ผลการทำนายเทียบกับค่าจริง
plt.figure(figsize=(6,4))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues',
            xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

# คำนวณและแสดงค่าความแม่นยำโดยรวม (Overall Accuracy)
accuracy = np.mean(y_pred.flatten() == y_test)
print(f"\nOverall Accuracy: {accuracy * 100:.2f}%")

ในส่วนของขั้นตอนนี้ทำหน้าที่  
- ใช้โมเดลที่ฝึกเสร็จแล้วในการ **ทำนายความน่าจะเป็น** ของรีวิวในชุดทดสอบ แล้วแปลงเป็นป้ายกำกับไบนารี (`0` = Negative, `1` = Positive) โดยใช้เกณฑ์ 0.5  
- สร้าง **รายงานการจำแนก (Classification Report)** ที่แสดงค่า Precision, Recall และ F1-score แยกตามแต่ละคลาส เพื่อประเมินประสิทธิภาพของโมเดลอย่างละเอียด  
- สร้าง **Confusion Matrix** ในรูปแบบ Heatmap ด้วยไลบรารี Seaborn เพื่อแสดงจำนวนการทำนายถูก/ผิดของแต่ละคลาส ช่วยให้มองเห็นจุดแข็งและจุดอ่อนของโมเดลได้ชัดเจน  
- คำนวณ **ความแม่นยำโดยรวม (Overall Accuracy)** โดยเปรียบเทียบค่าทำนายกับค่าจริงทั้งหมด แล้วแสดงผลเป็นเปอร์เซ็นต์

# ขั้นตอนที่ 6 ทดสอบและบันทึกโมเดล

In [ ]:
# สร้างฟังก์ชันสำหรับทำนายความรู้สึกจากข้อความใหม่ โดยผ่านขั้นตอนการทำความสะอาดและแปลงเป็นลำดับตัวเลข
def predict_sentiment(text):
    cleaned = clean_text(text)
    seq = tokenizer.texts_to_sequences([cleaned])
    pad = pad_sequences(seq, maxlen=MAX_LEN, padding='post', truncating='post')
    pred = model.predict(pad)[0][0]
    return "Positive" if pred > 0.5 else "Negative", float(pred)

# ทดสอบฟังก์ชัน predict_sentiment ด้วยตัวอย่างข้อความรีวิว 3 แบบ
print(predict_sentiment("This chocolate cake is absolutely delicious!"))
print(predict_sentiment("Tasteless and overpriced junk."))
print(predict_sentiment("It doesn’t taste good"))

# บันทึกโมเดลที่ฝึกเสร็จแล้วในรูปแบบไฟล์ .h5 ไปยัง Google Drive
model.save('/content/drive/MyDrive/food_review_sentiment_model.h5')

# บันทึก tokenizer ที่ใช้ในการฝึกโมเดลเป็นไฟล์ pickle เพื่อให้สามารถโหลดกลับมาใช้ในการ deploy ได้
with open('/content/drive/MyDrive/tokenizer_sentiment.pickle', 'wb') as f:
    pickle.dump(tokenizer, f)

# แสดงข้อความยืนยันการบันทึกสำเร็จ
print("\nบันทึกโมเดลและ tokenizer สำเร็จ!")

ในส่วนของขั้นตอนนี้ทำหน้าที่  
- สร้าง **ฟังก์ชัน `predict_sentiment`** ที่รับข้อความรีวิวใหม่ แล้วผ่านกระบวนการเดียวกับชุดฝึก: ทำความสะอาดข้อความ → แปลงเป็นลำดับตัวเลข → ปรับความยาว → ทำนายด้วยโมเดล → แปลงผลลัพธ์เป็นป้ายกำกับ (“Positive” หรือ “Negative”) พร้อมค่าความน่าจะเป็น  
- **ทดสอบฟังก์ชัน** ด้วยตัวอย่างข้อความจริง 3 ประโยค เพื่อตรวจสอบว่าโมเดลตอบสนองต่อรีวิวเชิงบวกและเชิงลบได้อย่างเหมาะสม  
- **บันทึกโมเดลที่ฝึกเสร็จแล้ว** ในรูปแบบไฟล์ `.h5` ไปยัง Google Drive เพื่อใช้ในขั้นตอนการ deploy (เช่น กับ FastAPI)  
- **บันทึก `tokenizer` ที่ใช้ในการฝึก** เป็นไฟล์ `pickle` ควบคู่กันไป เนื่องจาก tokenizer นี้มีพจนานุกรมเฉพาะที่โมเดลเรียนรู้มา จำเป็นต้องใช้ร่วมกันเมื่อประมวลผลข้อความใหม่  
- แสดงข้อความยืนยันว่าการบันทึกไฟล์ทั้งสองสำเร็จ พร้อมใช้งานในอนาคต